In [2]:
import os
import pandas as pd
import numpy as np

# Function to process a single trial and return the event, max, and min values
def process_trial(filename):
    # Read the CSV file
    data = pd.read_excel(filename, skiprows=8)

    # Identify events (assuming 'VEM' indicates events)
    event_indexes = data[data['VEM'] == 1].index

    # Create the 'phase' column and set values between 'VEM' events
    for i in range(1, len(event_indexes)):
        start = event_indexes[i - 1]
        end = event_indexes[i]
        data.loc[start:end, 'phase'] = i

    # Create an empty DataFrame to store the event, max, and min values
    event_values = pd.DataFrame()

    # Function to find maximum values for each phase
    def find_max_values(df, phase_col, phase_value):
        phase_data = df[df[phase_col] == phase_value]
        max_values = phase_data.iloc[:, 1:].max()
        return max_values

    # Function to find minimum values for each phase
    def find_min_values(df, phase_col, phase_value):
        phase_data = df[df[phase_col] == phase_value]
        min_values = phase_data.iloc[:, 1:].min()
        return min_values

    # Iterate through events (where 'VEM' equals 1) and add values to the event_values DataFrame
    for i, event_index in enumerate(event_indexes):
        event_data = data.iloc[event_index]
        event_values = event_values.append(event_data, ignore_index=True)

    # Find the maximum and minimum values between events and add them to the event_values DataFrame
    maximum_values_between_events = {}
    minimum_values_between_events = {}
    for i in range(1, len(event_indexes)):
        start = event_indexes[i - 1]
        end = event_indexes[i]
        values_max = data.iloc[start + 1 : end][data.columns[2:]].max()
        values_min = data.iloc[start + 1 : end][data.columns[2:]].min()
        maximum_values_between_events[f'Between Events {i}'] = values_max
        minimum_values_between_events[f'Between Events {i}'] = values_min

    return event_values, maximum_values_between_events, minimum_values_between_events

# Directory path where your trial files are located
directory = '/Users/brenthokeness/Desktop/PitchingData/PZ_FB'

# Get a list of filenames in the directory
filenames = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.xls')]

# Create a single Excel file to store all data
output_file = '/Users/brenthokeness/Desktop/PZ_FB_Analysis.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for filename in filenames:
        event_values, max_values_between_events, min_values_between_events = process_trial(filename)

        # Save the event values, maximum values, and minimum values in the same sheet
        max_min_df = pd.DataFrame()
        max_min_df = max_min_df.append(event_values)
        max_min_df = max_min_df.append(pd.DataFrame(max_values_between_events).T)
        max_min_df = max_min_df.append(pd.DataFrame(min_values_between_events).T)

        max_min_df.to_excel(writer, sheet_name=f'{os.path.basename(filename)}', index=False)


/var/folders/nf/5f05bjd51wxgmrxzcl0lxwl80000gn/T/ipykernel_97749/4066188083.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  event_values = event_values.append(event_data, ignore_index=True)
/var/folders/nf/5f05bjd51wxgmrxzcl0lxwl80000gn/T/ipykernel_97749/4066188083.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  max_min_df = max_min_df.append(event_values)
/var/folders/nf/5f05bjd51wxgmrxzcl0lxwl80000gn/T/ipykernel_97749/4066188083.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  max_min_df = max_min_df.append(pd.DataFrame(max_values_between_events).T)
/var/folders/nf/5f05bjd51wxgmrxzcl0lxwl80000gn/T/ipykernel_97749/4066188083.py:69: FutureWarning: The frame.append method is deprecated and will be removed from 